<img src="images/astradrel_datascience_logo.png" style="height: 100px;" align=left>
<img src="images/bs_logo.png" style="height: 100px;" align=right>

# Project: Web Scrapping Job Postings from Indeed Malaysia

Almost all of fresh graduates in Malaysia are using online job posting websites like **Indeed, LinkedIn, JobStreet** and likewise to look for jobs. So many opportunities exists on the internet yet only fraction of total fresh graduates will get the jobs which proves the competition existed in the online jobseeking. Many attributes of the candidates itself can affect their chances of landing their dream jobs such as skills, experiences, salary expectation and many more. 

In the case of Data Science, with the recent influx of working professional and fresh graduate alike from different background looking for a career transition this maybe become a trouble when it comes to meeting the criterias of the recruiter. With so many people competing for the same jobs posts, it become apparent that the recruiter needed to filter down the candidates and only takes the top of the cream among all of them. In this project we will gather several data science jobs posting from various online job posting website and determining what the desired attributes that recruiter looking for in a candidates.

In this project, we are using web scrapping tools to scrap the information of a jobs along with its jobs description, job employment type, salary, location and several more for **Data Scientist** position from the **Indeed Malaysia**.

# Module: Web Scrapping using BeautifulSoup

## Importing libraries

In [123]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import time
import string
import re

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
          'cookie':'CTK=1fhj1do4onpne802; indeed_rcc=CTK; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtd1A2MVASG5mFSBOh1iRBaM="; SESSION_START_TIME=1633800347801; SESSION_ID=1fhj1do4pnpne801; _ga=GA1.2.578343193.1633800352; SESSION_END_TIME=1633811136805; optimizelyEndUserId=oeu1635319128106r0.8882288061773578; _mkto_trk=id:699-SXJ-715&token:_mch-indeed.com-1635319130858-78920; _gcl_au=1.1.421520622.1635665164; LOCALE=en_MY; _gid=GA1.2.1217473174.1635865865; CSRF=EhuX1h0GXQRXj1hdWKBd1XPmfw955lYM; SURF=5gI0F8aGWBSDpOIqUS9Gt7bU5Icd5hiR; MICRO_CONTENT_CSRF_TOKEN=KbSgv993fSn3yrmpUgTkQdA0FbRZ3wSq; SHARED_INDEED_CSRF_TOKEN=hVEvDqD69V2AEUlPQMfu2Y7BqAQXiaHN; gonetap=2; g_state={"i_p":1636599041016,"i_l":3}; LC="co=MY"; _gat=1; _gali=jobDescriptionText'
          }

count = 1


## Initializing functions for web scrapping

In [112]:
# Initialize search query URL

def get_url(position, location):
    
    urltemplate = "https://malaysia.indeed.com/jobs?q={}&l={}"
    url = urltemplate.format(position, location)
    return url 

In [121]:
# Get the jobs details from the page

def fetch_jobs(post):
    
    ####################################################################################################
    # Get Job Title
  
    title = post.h2.findAll('span')
    
    for t in title:
        x = t.get('title')
        if x != None:
            job_title = x
        
    try:
        company = post.find('span','companyName').text
        job_company = company
            
    except AttributeError:
        job_company = ''

    ####################################################################################################
    # Get Job Location, Date, and URL
    
    job_location = post.find('div', 'companyLocation').text.replace('•Temporarily Remote','')
    job_date = post.find('span','date').text
    job_link = str("https://malaysia.indeed.com" + post['href'])
    
    ####################################################################################################
    # Get Job Salary

    try:
        salary = post.find('div','metadata salary-snippet-container').text
        job_salary = salary
        
    except AttributeError:
        job_salary = 'No Detail'
    
    ####################################################################################################
    # Get Job Description
    response = requests.get(job_link, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    try:
        job_description = soup.find('div','jobsearch-jobDescriptionText').text
        
    except AttributeError:
        job_description = 'No Details'
        
    ####################################################################################################  
    # Get Job Type
    
    try:
        job_type = soup.find('span', 'jobsearch-JobMetadataHeader-item').text
        job_type = "".join(l for l in job_type if l not in string.punctuation).strip()
    
    except AttributeError:
    
        try:
            job_type = str(soup.find_all('p'))
            pattern = r"(?<=Job Type: )[^<:]+(?=:?<)"
            job_type = re.search(pattern,job_type).group().replace("-"," ")
            
    
        except AttributeError:
            job_type = "No Detail"
        
    ####################################################################################################   
       
    record = [job_title, job_company, job_location, job_salary, job_date, job_link, job_type, job_description]
    return record
    
    

In [114]:
# get next page

def scrap_indeed():
    


    job_records = []
    
    a = input("Please enter what kind of job are you looking for?: ")
    b = input("Please enter where the job are located at?: ")
    
    name = str(a).strip().replace(" ", "%20")   
    location = str(b).strip().replace(" ", "%20")   
    url = get_url(name, location)
    
        
    
    while True:
        
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        posts = soup.find_all('a', 'tapItem')
        for post in posts:
            job_records.append(fetch_jobs(post))
       
            
        try:
            next_page = soup.find('a',{'aria-label': 'Next'}).get('href')
            url = "https://malaysia.indeed.com" + next_page
            
        
        except AttributeError:
            break   
    return job_records
    

## Scrapping job postings from the website 

In [115]:
soup

<html>
<head>
<title>hCaptcha solve page</title>
<script async="" defer="" src="https://www.hcaptcha.com/1/api.js"></script>
</head>
<body>
<form action="/jobs?q=Data%20Scientist&amp;l=Malaysia" method="POST">
<div class="h-captcha" data-sitekey="eb27f525-f936-43b4-91e2-95a426d4a8bd"></div>
<br/>
<input type="submit" value="Submit"/>
</form>
</body>
</html>

In [116]:
# Scrap all the jobs available from Indeed.com

indeed_records = scrap_indeed()

Please enter what kind of job are you looking for?: Data Scientist
Please enter where the job are located at?: Malaysia
['Meteorologist/ Meteorological Data Scientist', 'AkiraKan (Marine Technology) Sdn Bhd', 'Kuala Lumpur', 'RM 3,500 a month', '13 days ago', 'https://malaysia.indeed.com/company/AkiraKan-(Marine-Technology)-Sdn-Bhd/jobs/Meteorologist-1d734af444538030?fccid=dad36d3ea9a30fd9&vjs=3', 'Full time', 'AkiraKan [ AKN Technologies ] is hiring Meteorologists. We work with the oil and gas industry and agriculture industry to provide weather forecasts for the weather sensitive operations. We also conduct state-of-the-art weather research to improve the weather forecast accuracy and systems.You will work with experienced meteorologists, data scientists, data engineers, machine learning engineers, and project managers to improve next generation weather forecasting technologies.Responsibilities: As a meteorologist, you will be supporting two main areas; forecasting and research.Invol

['Geomatics Surveyor (Geographic Information System)', 'AkiraKan (Marine Technology) Sdn Bhd', 'Kuala Lumpur', 'RM 3,500 a month', '13 days ago', 'https://malaysia.indeed.com/company/AkiraKan-(Marine-Technology)-Sdn-Bhd/jobs/Geomatic-Surveyor-68a65bfad29aed19?fccid=dad36d3ea9a30fd9&vjs=3', 'Full time', 'AkiraKan [ AKN Technologies ] is hiring surveyor/GIS Expert/cartographer to join our engineering teams in our Penang/KL office.We are developing a comprehensive HD map to improve transit, land transportation and marine operation. You will work with other data scientists, data engineers, machine learning engineers, designers and project managers to improve next generation mapping technologies.Responsibilities: Capture, process and publish Lidar and Photogrammetry data using various GIS ToolsUtilize various software packages and information from various sources to extract and create GIS dataEnsure data accuracy, quality & completenessAcquire, georeferenced, digitize and perform QA/QC on d

['Consultant/Senior Consultant - Data Scientist - MY', 'Deloitte', 'Kuala Lumpur', 'No Detail', '5 days ago', 'https://malaysia.indeed.com/rc/clk?jk=9d4796e2908ca32e&fccid=9e215d88a6b33622&vjs=3', 'No Detail', 'Consultant/Senior Consultant - Data Scientist - MY\nDate: 29-Oct-2021\nService Line / Portfolios: Strategy, Growth & Innovation\nLocation:\nKuala Lumpur, MY\nConsultant/Senior Consultant - Data Scientist - SEA\n\nAt Deloitte, we offer a unique and exceptional career experience to inspire and empower talents like you to make an impact that matters for our clients, people and community. Deloitte offers you a highly inclusive, collaborative workplace and unrivalled opportunities to realize your full potential. We are always looking for people with the relentless energy to push themselves further, and to find new avenues and unique ways to reach our shared goals.\n\nWork you will do\n\nThis role sits within the Analytics and Cognitive team of Deloitte Southeast Asia, which encompass

['Data Scientist', 'RHB Banking Group', 'Kuala Lumpur', 'No Detail', '22 days ago', 'https://malaysia.indeed.com/rc/clk?jk=99fddd747f862790&fccid=fffa4d14a56af74d&vjs=3', 'No Detail', 'Working Hour\nRegular Hours\nMonday - Friday\nBusiness Area\nStrategy & Analytics\nLocation\nMalaysia - Kuala Lumpur\nDescription\n\nPrimary Objective:\nTo participate and contribute in identification of business use-cases for innovative ideation, design, build, test, and execution for Group Retail Banking/Group Business Banking business units to uplift business revenue or in cost optimization efforts\nTo develop and deliver business use-cases for innovative ideation, design, build, test, and execution for GRB / GBB business units to uplift business revenue.\nTo execute and roll-out the Identified ideas/projects with Business Justifications (with P & L sizing impact) and work with Stakeholders on ensuring these opportunities are tracked and managed\nKey Responsibilities:\nTo engage with respective Retail

['Data Scientist', 'ASK Resources', 'Kuala Lumpur', 'No Detail', '2 days ago', 'https://malaysia.indeed.com/rc/clk?jk=90066c2e848ea7a0&fccid=193c4dc64bfdba42&vjs=3', 'No Detail', "Company Profile:\nOverseas Healthcare Company\n\nPosition:\nData Scientist\n\nLocation:\nNearby Cheras area\n\nJob Scopes:\n\nAccountable for Data model creation, development, visualization, deployment and optimization process.\nDevelop and integrate models for smart devices and provide support on user acceptance test.\nCreate & execute the data science approach and roadmap to support business needs.\nPerform the entire cycle of machine learning process and get predicted results for better business insight.\nUnderstand the algorithms / methodologies related implications of advanced analytics solutions; and assist in defining the solutions to related businss problem.\nSupport any-hoc data science projects as assigned.\n\nRequirements:\n\nMinimum Degree in Computer Science, Software Engineering or equivalent\nP

...

['Senior Manager, Performance & Data Analytics', 'Prudential', 'Kuala Lumpur', 'No Detail', '7 days ago', 'https://malaysia.indeed.com/rc/clk?jk=9e166b5a8407665b&fccid=4144c0c6235e4037&vjs=3', 'No Detail', 'At Prudential, we understand that success comes from the talent and commitment of our people. Together, we have a shared vision in securing the future of our customers and our communities. We strive to build a business that you can shape, an inclusive workplace where everyone’s ideas are valued and a culture where we can thrive together. Our people stay connected and tuned in to what’s happening around us, keeping us ahead of the curve. While focused on the long-term, we look to the future to bring growth, development and benefit to everyone whose lives we touch.\nJOB SCOPE:\n\n\nDrive long term sustainable portfolio.\nDrive efficiency and exceptional customer experience by creating insights through Data\nanalytics (Internal & External Data).\n\nUtilize AI & analytics to create oppo

['Product Designer', 'Fave Group Pte Ltd', 'Kuala Lumpur', 'No Detail', '30+ days ago', 'https://malaysia.indeed.com/rc/clk?jk=3d7eb5ed50f9d253&fccid=56bc44b283400bfd&vjs=3', 'No Detail', "Department\nProduct Design\nJob posted on\nJun 6, 2021\nEmployment type\nFull Time\nExperience range\n0 - N.A.\nFave is one of Southeast Asia's leading fintech platforms; providing a smart payment app for the smart generation of consumers to pay & save. Fave also empowers offline businesses to grow and digitally connect with their customers in a whole new way. Fave is on a mission to maximise joy & value in every shopping experience and accelerate the offline world’s transition to the digital economy.\nFave currently operates in 35 cities across Malaysia, Singapore, and Indonesia. Fave will be launching in India in 2021.\nIf being a part of a digital revolution in the fastest-growing region in the world excites you, get in touch today!\nProduct Design at Fave\nWe don’t follow the traditional corporat

['Lead Data Engineer', 'AirAsia', 'Brickfields', 'No Detail', '30+ days ago', 'https://malaysia.indeed.com/rc/clk?jk=c5e28cd091fdc7c9&fccid=350f4541eb63ccf6&vjs=3', 'No Detail', "Job Description\nWhy AirAsia?\nAre you ready to take off and be part of the Allstar employee? Whether you’re applying for a developer, customer happiness or crew, at AirAsia we act as One AirAsia.\nIf you are hungry to make a difference with one of the most well known low-cost airlines and to work in the dynamic technology hub, this is the job for you.\nAirline Technology Data team drives conversion, personalisation, and revenue growth across flights and other lines of business. We are looking for Allstars to join an innovative and dynamic team which works in tandem with product and tech. You will help develop the future models and tools to drive better decisions and growth.\nWhat You'll Do:\nAssist in creating a state of the art real time event streaming infrastructure\nLead the projects end-to-end and, if re

['Automation Test Engineer, AI', 'MoneyLion', 'Malaysia', 'No Detail', '1 day ago', 'https://malaysia.indeed.com/rc/clk?jk=47488470b1a052d2&fccid=f0df775ddffb40cc&vjs=3', 'No Detail', "Automation Test Engineer, AI\nAbout Us\nMoneyLion is America's most powerful and rewarding financial membership, empowering people to take control of their finances to achieve their overall life goals.\nSince its launch in 2013, MoneyLion has amassed more than 4 million members and has earned its customers trust with new banking and investments capabilities that offer a one-of-a-kind rewards program, no-fee checking account, overdraft protection, and loans funded in 15 seconds or less.\nFrom a single application platform, users can get a holistic snapshot of their entire financial lives while having access to personalized and contextualized tips and tools to improve their credit and achieve everyday savings.\nBy rewarding financial responsibility, providing top-notch customer assistance and offering insi

['Credit Decision Science Specialist', 'Hong Leong Bank', 'Kuala Lumpur', 'No Detail', '30+ days ago', 'https://malaysia.indeed.com/rc/clk?jk=bdbd026c755ff057&fccid=59850394de459c2b&vjs=3', 'No Detail', "If you are looking to excel and make a difference, take a closer look at us…\nThis position has the full responsibility for the teams covering end to end development of Credit Risk Models covering all unsecured and secured lending products across the different markets. We are seeking motivated and skilled professional with strong business acumen with hands on approach to complete assigned tasks within tight deadlines. The candidate must be a team player and result oriented, poses excellent communication and presentation skills, and able to work under tight timeline.\nKey Responsibilities:\nFull responsibility for the teams covering end to end development of Credit Risk Models covering all unsecured and secured lending products.\nOptimizing the risk/return outcomes through creation of d

In [117]:
len(indeed_records)

150

In [119]:
# Creating DataFrame for Indeed Job Posting

pd.set_option('display.max_colwidth', None)
columns=['Job Title','Company','Location', 'Salary', 'Post Date', "Job Link", "Type", "Description"]
df_indeed = pd.DataFrame(indeed_records, columns=columns)
df_indeed

Job Title  \
0          Meteorologist/ Meteorological Data Scientist   
1                                        Data Scientist   
2                                        Data Scientist   
3                                        Data Scientist   
4    Geomatics Surveyor (Geographic Information System)   
..                                                  ...   
145                  Credit Decision Science Specialist   
146                                     Product Manager   
147                                    Product Designer   
148                        Regulatory Officer/Executive   
149                            PRODUCT OWNER (Contract)   

                                  Company      Location  \
0    AkiraKan (Marine Technology) Sdn Bhd  Kuala Lumpur   
1             Doo Technology MY Sdn. Bhd.  Kuala Lumpur   
2               Datalabs Asia (M) Sdn Bhd  Kuala Lumpur   
3                       BASF Asia Pacific  Kuala Lumpur   
4    AkiraKan (Marine Technology) Sdn Bhd  Kuala Lumpur   
..                                    ...           ...   
145                       Hong Leong Bank  Kuala Lumpur   
146              Supahands Dotcom Sdn Bhd  Kuala Lumpur   
147              Supahands Dotcom Sdn Bhd  Kuala Lumpur   
148                       Bright Prospect         Johor   
149                               Kinesso  Kuala Lumpur   

                          Salary     Post Date  \
0               RM 3,500 a month   13 days ago   
1    RM 6,000 - RM 7,999 a month    6 days ago   
2    RM 5,000 - RM 5,999 a month   13 days ago   
3                      No Detail   15 days ago   
4               RM 3,500 a month   13 days ago   
..                           ...           ...   
145                    No Detail  30+ days ago   
146                    No Detail   24 days ago   
147                    No Detail   24 days ago   
148  RM 3,000 - RM 6,000 a month  30+ days ago   
149                    No Detail   15 days ago   

                                                                                                                                          Job Link  \
0        https://malaysia.indeed.com/company/AkiraKan-(Marine-Technology)-Sdn-Bhd/jobs/Meteorologist-1d734af444538030?fccid=dad36d3ea9a30fd9&vjs=3   
1                                                              https://malaysia.indeed.com/rc/clk?jk=e9d15ef56f089707&fccid=0447500ab5f8dca0&vjs=3   
2                                                              https://malaysia.indeed.com/rc/clk?jk=4b53359d43ff976d&fccid=e198b5a199f648cf&vjs=3   
3                                                              https://malaysia.indeed.com/rc/clk?jk=3d272182715dbaf6&fccid=38313780e9822361&vjs=3   
4    https://malaysia.indeed.com/company/AkiraKan-(Marine-Technology)-Sdn-Bhd/jobs/Geomatic-Surveyor-68a65bfad29aed19?fccid=dad36d3ea9a30fd9&vjs=3   
..                                                                                                                                             ...   
145                                                            https://malaysia.indeed.com/rc/clk?jk=bdbd026c755ff057&fccid=59850394de459c2b&vjs=3   
146                                                            https://malaysia.indeed.com/rc/clk?jk=31607d8273d0f4c0&fccid=ae5413ece72324d2&vjs=3   
147                                                            https://malaysia.indeed.com/rc/clk?jk=c78c82e78828987d&fccid=ae5413ece72324d2&vjs=3   
148                                                            https://malaysia.indeed.com/rc/clk?jk=e834561314c819d9&fccid=0987a8900dda09a1&vjs=3   
149                                                            https://malaysia.indeed.com/rc/clk?jk=be44416956d0d528&fccid=2e7fb2f475ab63dd&vjs=3   

          Type  \
0    Full time   
1    No Detail   
2     Contract   
3    No Detail   
4    Full time   
..         ...   
145  No Detail   
146  No Detail   
147  No Detail   
148  No Detail   
149  No Detail   

         

## Exporting the dataframe created 

In [120]:
# Exporting the dataframe to csv format

df_indeed.to_csv(r'data/indeed_jobs.csv', index = False)

In [122]:
# Exporting the dataframe to excel format

df_indeed.to_excel(r'data/indeed_jobs.xlsx', index = False)